# Der Dschanibekow-Effekt

Letzte Woche haben wir begonnen, den sogenannten Dschanibekow-Effekt zu untersuchen.
Hier nochmal, was die Wikipedia-Seite dazu sagt:

"Der Dschanibekow-Effekt, auch Tennisschlägereffekt, ist eine besondere Form des Torkelns eines kräftefrei rotierenden Körpers, die 1985 vom sowjetischen Kosmonauten Wladimir Dschanibekow während eines Raumfluges an einer Flügelmutter beobachtet wurde.

Grundsätzlich ist seit 1834 bekannt, dass ein frei rotierender Körper mit drei unterschiedlichen Hauptträgheitsmomenten eine stabile Orientierung der Drehachse nur zeigt, wenn er näherungsweise um eine der beiden Hauptträgheitsachsen rotiert, zu denen das größte bzw. das kleinste Trägheitsmoment gehört. Bei Rotation um die dritte, dazu senkrechte Hauptträgheitsachse hingegen entwickelt der Körper aus kleinsten Abweichungen große Torkelbewegungen, wenn der Drehimpulsvektor anfangs nicht exakt mit dieser Hauptträgheitsachse übereinstimmt. Der Drehimpulsvektor selbst bleibt dabei konstant, nicht aber die Richtung der momentanen Drehachse in Bezug auf das körperfeste und das raumfeste Koordinatensystem.

Die besonders eigentümliche Art des Torkelns, die Dschanibekow in der Schwerelosigkeit beobachtete, scheint sogar eine wiederholte Umkehr der Drehrichtung einzuschließen (wie im Video zu sehen, siehe unten). Diese Umkehrung gilt aber lediglich in Bezug auf das körperfeste Koordinatensystem, während vom raumfesten System aus gesehen der Drehsinn sich nicht umkehrt.

Diese Bewegung wurde 1991 theoretisch begründet. Der Effekt beruht in mathematischer Sicht darauf, dass die betreffende Hauptträgheitsachse auf dem Energieellipsoid nicht wie die anderen beiden Achsen zu einem elliptischen, sondern zu einem hyperbolischen Fixpunkt gehört (genauer: zu einem Sattelpunkt); Näheres siehe Bewegung kräftefreier Kreisel."

Hier wollen wir nun basierend auf dem Jupyter-Notebook der letzten Woche den Trägheitstensor eines T-förmigen Körpers mithilfe numerischer Monte-Carlo-Integration bestimmen und dann damit die Bewegungsgleichungen integrieren.

In [1]:
# Zunächst soll der Effekt anhand eines Videos von der Wikipedia-Seite illustriert werden:
from IPython.display import HTML

# Groesse des Videos
width = 800
height = 450
frac = 0.75

HTML("""
<iframe src="//commons.wikimedia.org/wiki/File:Dzhanibekov_effect.ogv?embedplayer=yes" width="{}" height="{}" frameborder="0" webkitAllowFullScreen mozallowfullscreen allowFullScreen></iframe>
""".format(int(width * frac),int(height * frac)))

Um solch komplexes Verhalten zu verstehen, beschreiben wir den Körper als Starren Körper. Für die Bewegungsgleichungen wird dazu der Trägheitstensor benötigt. 

Die Berechnung von Trägheitstensoren mittels numerischer Integration soll im Folgenden behandelt werden.

# Trägheitstensoren mithilfe numerischer Monte-Carlo Integration berechnen

In [ ]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate

from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

Zur Wiederholung und der Vollständigkeit halber hier nochmal die Schritte des letzten Notebooks:

Die Integration soll durch ein sogenanntes Monte-Carlo-Verfahren durchgeführt werden, bei der zufällig ein Punkt im Raum bestimmt wird. Liegt dieser Punkt innerhalb des Körpers, dann ergibt dies einen Beitrag zum Integral, wenn nicht, dann ist der Beitrag null. Der Computer muss dazu mithilfe von Zufallszahlen solche Raumpunkte bestimmen, dazu werden sogenannte Zufallszahlengeneratoren genutzt. Im Folgenden werden Sie durch die wesentlichen Schritte geführt. Wie sonst auch, müssen Sie die mit ??? markierten Stellen ergänzen.

Um eine einheitliche Beschreibung verschiedener Körper zu gewährleisten, nutzen wir hier ein Programmierkonzept aus der sogenannten 'Objektorientierten Programmierung', sogennante 'Klassen'.

Zur Beschreibung verschiedener Körper implementieren wir die Klasse `starrer_koerper`. Klassen sind ein fortgeschrittenes Konstrukt, welches in vielen Programmiersprachen vorhanden ist und zum Beispiel in physikalischen Anwendungen sehr natürlich Anwendung findet. 

In diesem Notebook implementieren wir zum Beispiel die `scatter` Methode für `starrer_koerper`, die dann zur Behandlung jedes Starren Körpers, der der Klasse bekannt ist, genutzt werden kann.

In [ ]:
class starrer_koerper:
    """Dummy Object for geometric figures"""
    def __init__(self):
        """Legt die geometrischen Eigenschaften fest"""
        pass
        
    def __call__(self,x):
        """Soll fuer die Integration genutzt werden
            return 0 falls auserhalb Object
            return 1 falls innerhalb Object     
        """
        pass
    
    def scatter(self,n_pts, lim = 1):
        """Supposed to give an idea of the shape of the object"""
        pts = np.random.random((n_pts,3))*2*lim - lim
        hit = self(pts)
        inds = np.where(hit == 1)
        points = pts[inds]
        # Create the figure
        fig = plt.figure(figsize=(6,5))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(points[...,0],points[...,1],points[...,2], c = 'b', marker='o')
        ax.set_xlim(-lim,lim)
        ax.set_ylim(-lim,lim)
        ax.set_zlim(-lim,lim)
        plt.show()

## Monte-Carlo-Integration

Die Integration soll mithilfe eines Monte-Carlo-Verfahrens durchgeführt werden. Das Konzept der Punktwolke, die wir oben betrachtet haben, ist dabei bereits sehr illustrativ.

Zum Beispiel kann auf diese Weise das Volumen des Körpers recht einfach bestimmt werden: es werden zufällig Punkte im Raum bestimmt ('ausgewürfelt' wie beim Glücksspiel in einem Casino in Monte Carlo) und dann gezählt, wie viele dieser Punkte innerhalb und wie viele ausserhalb des Körpers liegen.
Zur Bestimmung eines Integrals, wie zur Berechnung des Trägheitstensors benötigt, muss noch die Funktion am zufällig bestimmten Raumpunkt ausgewertet werden, falls dieser innerhalb des Integrationsvolumens (hier der von uns betrachtete Körper) liegt. 

Mehr Informationen gibt es zum Beispiel unter
https://en.wikipedia.org/wiki/Monte_Carlo_integration

In [ ]:
def momentum_tensor(body,n_pts,lim = 2):
    pts = np.random.random((n_pts,3))*2*lim - lim
    I = np.empty((3,3))
    hit = body(pts)
    
    # Integrationen
    I[0,0] = ((pts[...,1]**2 + pts[...,2]**2) * hit).mean()
    I[1,1] = ((pts[...,0]**2 + pts[...,2]**2) * hit).mean()
    I[2,2] = ((pts[...,0]**2 + pts[...,1]**2) * hit).mean()
    I[0,1] = I[1,0] = -((pts[...,0] * pts[...,1]) * hit).mean()
    I[0,2] = I[2,0] = -((pts[...,0] * pts[...,2]) * hit).mean()
    I[1,2] = I[2,1] = -((pts[...,1] * pts[...,2]) * hit).mean()
    
    # Wir benötigen zusätzlich die Massendichte, die wir ebenfalls mit den oben
    # 'gezogenen' Punkten Abschätzen:
    V   = hit.sum() / n_pts * (2*lim)**3
    rho = body.mass / V
    
    # Für das Integral Estimate fügen wir die Dichte hinzu und multiplizieren mit dem 
    # Volumen des gesammten Raumes in dem wir Punkte generiert haben
    return rho*I*(2*lim)**3

# Berechnung des T-förmigen Körpers (T-handle)

Wir wenden nun das Monte-Carlo-Verfahren an, um den Trägheitstensor des T-förmigen Körpers aus dem Video zu berechnen.

In [ ]:
class t_handle(starrer_koerper):
    def __init__(self,r1,r2,l1,l2, mass = 1):
        self.r1 = r1
        self.r2 = r2
        self.mass = mass
        self.l1 = l1
        self.l2 = l2
        
    def __call__(self,x):
        ret = np.zeros_like(x[...,0])
        
        # Wir betrachten den Körper als aus zwei orthogonal zueinander stehenden Zylindern zusammengesetzt.
        # Punkt im ersten Zylinder
        ???

        # Punkt im zweiten Zylinder
        ???
        
        return ret

In [ ]:
body = t_handle(0.5,0.4,4,3)
body.scatter(int(1e4),3)

In [ ]:
T = momentum_tensor(body,int(1e7),lim = 2)

Im Folgenden benötigen wir die zugehörigen Eigenwerte bzw. Eigenvektoren des Trägheitstensors. Diese können mit `np.linalg.eig(T)` bestimmt werden.

In [ ]:
l,v = np.linalg.eig(T)
l,v

Wir können uns das Leben hier etwas leichter machen. Physikalisch ist bei diesem geometrischen Objekt klar, dass die Eigenvektoren (diese sind die Hauptträgheitsachsen) mit den kartesischen Koordinatenachsen übereinstimmen müssen. Die numerisch bestimmten Eigenvektoren stimmen allerdings nicht genau mit diesen überein, was auf numerische Fehler zurückzuführen ist. 

Da die Bewegungsgleichungen allerdings über Rotationen um die Hauptträgheitsachsen definiert sind, vereinfacht sich die Betrachtung deutlich, wenn wir die numerischen Artifakte ignorieren, um dann nur noch Rotationen um die Koordinatenachsen zu betrachten.

Dies geschieht in den folgenden Zeilen, in denen 'nicht passende' Werte ausgefiltert werden, so dass die Einträge des Trägheitstensors, von denen wir erwarten, dass sie null sind, auf null gesetzt werden (vergleichen Sie mit dem Ergebnis oben, wo dies nicht gemacht wurde - von welcher Größenordnung ist der numerische Fehler, der dabei unterdrückt wird?):

In [ ]:
# Im allgemeinen ist das wahrscheinlich keine gute Idee
# Hier nur möglich aufgrund des Wissens über die Hauptträgheitsachsen
inds = np.where(np.isclose(v,0,atol=1e-2) == True)
v[inds] = 0
l,v

# Numerische Lösung der Bewegungsgleichungen

Die Bewegungsgleichungen für die Bewegungs eines starren Körpers im körpereigenen Bezugssystem sind die aus der Vorlesung bekannten Euler-Gleichungen:

\begin{align}
I_1 \dot \omega_1 &= (I_2 - I_3) \omega_2\omega_3 \\
I_2 \dot \omega_2 &= (I_3 - I_1) \omega_3\omega_1 \\
I_3 \dot \omega_3 &= (I_1 - I_2) \omega_1\omega_2 \, ,
\end{align}

wobei $I_j$ die jeweiligen Trägheitsmomente und $\omega_j$ die Winkelgeschwindigkeiten sind. Prinzipiell ist die Dynamik des starren Körpers damit beschrieben. Da wir am Ende die Rotation im externen Bezugssystem betrachten wollen integrieren wir noch die Winkel auf:

\begin{align}
\dot\theta_1 &= \omega_1\\
\dot\theta_2 &= \omega_2\\
\dot\theta_3 &= \omega_3\, .
\end{align}

Diese DGL werden nun numerisch gelöst. Das Vorgehen ist dabei analog zum Vorgehen im Notebook `AM_EX9.ipynb`: zunächst implementieren wir die entsprechenden Ableitungen und benutzen dann die von Python zur Verfügung gestellte `odeint`-Methode.

In [ ]:
# Reihenfolge der Koordinaten die Später genutzt werden sollen
# Koordinaten: theta1,theta2,theta3,omega1,omega2,omega3

def derivs(state,t):
    w1 = state[3]
    w2 = state[4]
    w3 = state[5]
    
    dydx    = np.zeros_like(state) 
    dydx[0] = ???
    dydx[1] = ???
    dydx[2] = ???
    dydx[3] = ???
    dydx[4] = ???
    dydx[5] = ???

    return dydx

# Wie verhalten sich die Winkelgeschwindigkeiten $\omega$ für eine Rotation um genau eine der Achsen?

In [ ]:
fig,ax = plt.subplots(3,1,figsize = (10,6),sharex=True)

for i in range(3):
    w0 = 3*[0]
    w0[i] = np.radians(20)
    state0 = [0,0,0] + w0
    
    dt = 0.1 
    t = np.arange(0.0, 1000, dt) 
    n_trj = integrate.odeint(derivs, state0, t) 
    
    ax[i].plot(t,n_trj[...,3],label=r'$\omega_1$')
    ax[i].plot(t,n_trj[...,4],label=r'$\omega_2$')
    ax[i].plot(t,n_trj[...,5],label=r'$\omega_3$')
    ax[i].set_xlim(t[0],t[-1])
    ax[i].legend(loc = "upper right")

plt.show()

## Jetzt fügen wir eine kleine Störung hinzu

In [ ]:
fig,ax = plt.subplots(3,1,figsize = (10,6),sharex=True)
distortion = np.radians(0.01)

for i in range(3):
    w0 = [distortion,distortion,distortion]
    w0[i] = np.radians(20)
    state0 = [0,0,0] + w0
    
    dt = 0.1 
    t = np.arange(0.0, 1000, dt) 
    n_trj = integrate.odeint(derivs, state0, t) 
    
    ax[i].plot(t,n_trj[...,3],label=r'$\omega_1$')
    ax[i].plot(t,n_trj[...,4],label=r'$\omega_2$')
    ax[i].plot(t,n_trj[...,5],label=r'$\omega_3$')
    ax[i].set_xlim(t[0],t[-1])
    ax[i].legend(loc = "upper right")

plt.show()

# Video

Für das rotierende Bezugssystem benötigen wir die Möglichkeit beliebig um einen Vektor zu rotieren, dazu implementieren wir die folgende [Rotationsmatrix](https://en.wikipedia.org/wiki/Rotation_matrix#Rotation_matrix_from_axis_and_angle):

In [ ]:
# Gibt die Rotationsmatrix um einen beliebigen Vektor

# Levi-Cevita Symbol für Indizes 0,1,2
eijk = np.zeros((3, 3, 3))
eijk[0, 1, 2] = eijk[1, 2, 0] = eijk[2, 0, 1] = 1
eijk[0, 2, 1] = eijk[2, 1, 0] = eijk[1, 0, 2] = -1

def rot(vec,theta):
    # Vektor normieren
    u = vec/np.linalg.norm(vec)
    
    # Rotationsmatrix zusammenbauen
    R = np.empty((3,3))
    for j,k in np.ndindex(3,3):
        if j==k:
            R[j,k] = np.cos(theta/2)**2 + np.sin(theta/2)**2 * (2*u[j]**2-1)
        else:
            R[j,k] = 2*u[j]*u[k]*np.sin(theta/2)**2 - (eijk[j,k,...]*u).sum() * np.sin(theta)
    return R    

In [ ]:
# Wiederholen der numerischen Berechnung für den Interessanten Fall,
# bei mir ist das für Rotationen um die "y"-Achse
distortion = np.radians(0.001)
w0 = [distortion,0,0]
w0[1] = np.radians(40)
state0 = [0,0,0] + w0

dt = 0.01 
t = np.arange(0.0, 1000, dt) 
n_trj = integrate.odeint(derivs, state0, t) 

In [ ]:
fig = plt.figure(figsize = (9,8))
ax = fig.gca(projection='3d')

# Setzen der Plot limits
lim = 1
ax.set_xlim(-lim,lim)
ax.set_ylim(-lim,lim)
ax.set_zlim(-lim,lim)

# Erstellen eines Referenz-Systems
xyz = np.empty((3,3))
xyz[0] = np.array([1,0,0])
xyz[1] = np.array([0,1,0])
xyz[2] = np.array([0,0,1])

# Erstellen der Elemente im Plot
lines   = [ax.plot([], [], 'o-', lw=2)[0] for _ in range(3)]

# Initialisierung
def init():
    for line in lines:
        line.set_data([],[])
    return lines

# Wir überspringen einige Zeitpunkte in der Erstellung des Videos
# was dazu führt, dass man etwas mehr im Video sieht
offs = 40

# Animationsfunktion, die für jedes Frame ausgeführt wird
def animate(i):
    
    # Jetzt kommen die aufintegrierten Winkel ins Spiel
    # Für jeden Zeitpunkt wird das Referenz-Koordinatensystem um die Differenz der Winkel rotiert
    for j in range(offs):
        for k in range(3):
            mat = rot(xyz[k],(n_trj[i*offs+1+j,k] - n_trj[i*offs+j,k]))
            xyz[0] = xyz[0].dot(mat)
            xyz[1] = xyz[1].dot(mat)
            xyz[2] = xyz[2].dot(mat) 
    
    # xyz entspricht jedem dem rotierten Referenzsystem zum entsprechenden Zeitpunkt
    # Bei meiner Konstruktion des T-Stücks liegen das obere Stück des T auf der x-Achse
    #     und der Fuß auf der y-Achse
    # Wenn man sich mehr Mühe geben möchte sollte man die Parameter aus t_handle auslesen und das T
    #     mit den richtigen Längen zeichnen
    lines[0].set_data([0,xyz[1,0]],[0,xyz[1,1]])
    lines[0].set_3d_properties([0,xyz[1,2]]) 
    lines[1].set_data([0,xyz[0,0]],[0,xyz[0,1]])
    lines[1].set_3d_properties([0,xyz[0,2]]) 
    lines[2].set_data([0,-xyz[0,0]],[0,-xyz[0,1]])
    lines[2].set_3d_properties([0,-xyz[0,2]]) 
    return lines

# Erzeugt die Animation
ani = animation.FuncAnimation(fig, animate, np.arange(1, len(t)//offs//10),interval=1000*dt/3*offs/3, blit=False, init_func=init) 
plt.close(ani._fig)

In [ ]:
plt.rcParams['animation.embed_limit'] = 40 # etwas mehr video, kann aber auch auskommentiert werden
%time HTML(ani.to_jshtml())